# 2022-09-11 • Fit function to STA

## Imports

In [3]:
#

In [4]:
using Revise

In [5]:
using MyToolbox

In [7]:
using VoltoMapSim

## Params

In [8]:
p = get_params(
    duration = 10minutes,
    p_conn = 0.04,
    g_EE = 1,
    g_EI = 1,
    g_IE = 4,
    g_II = 4,
    ext_current = Normal(-0.5 * pA/√seconds, 5 * pA/√seconds),
    E_inh = -80 * mV,
    record_v = [1:40; 801:810],
);

## Run sim

In [9]:
s = cached(sim, [p.sim]);

┌ Warning: read type NamedTuple is not a leaf type in workspace; reconstructing
└ @ JLD2 C:\Users\tfiers\.julia\packages\JLD2\k9Gt0\src\data\reconstructing_datatypes.jl:237


In [11]:
snt = s.data;

(num_timesteps = 6000000, timesteps = 0:0.0001:600, neuron_IDs = (exc = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  791, 792, 793, 794, 795, 796, 797, 798, 799, 800], inh = [801, 802, 803, 804, 805, 806, 807, 808, 809, 810  …  991, 992, 993, 994, 995, 996, 997, 998, 999, 1000]), synapse_IDs = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  39585, 39586, 39587, 39588, 39589, 39590, 39591, 39592, 39593, 39594], ODE_var_IDs = (t = 1, v = (exc = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11  …  792, 793, 794, 795, 796, 797, 798, 799, 800, 801], inh = [802, 803, 804, 805, 806, 807, 808, 809, 810, 811  …  992, 993, 994, 995, 996, 997, 998, 999, 1000, 1001]), u = (exc = [1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011  …  1792, 1793, 1794, 1795, 1796, 1797, 1798, 1799, 1800, 1801], inh = [1802, 1803, 1804, 1805, 1806, 1807, 1808, 1809, 1810, 1811  …  1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001]), g_exc = (exc = [2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011  …  2792, 2793, 2794, 2795, 2796

In [7]:
s = augment(s, p);

## ...

From previous notebook, median input detection rates, for both our old connection test technique (peak-to-peak and area-over-start for exc/inh decision) and the new one, where we cheated with info of the average E→E STA:

```
            ptp-area  corr-with-avgSTA 
──────────────────────────────────────
 exc → exc      0.57              0.79
 exc → inh      0.57              0.82
 inh → exc      0.88              1.00
 inh → inh      0.81              0.96
```

If we don't cheat, but still use the "template"- or "canonical STA shape"-matching idea, can we still beat the `ptp-area` detection rates?

From the "Model PSP" section [here](https://tfiers.github.io/phd/nb/2022-09-05__Why-not-detected.html#model-psp), we have a functional form for a PSP.

..But not for an STA, which shows an additional dimple.  

Let's not research additional theoretical justification for the STA shape (something to do with spike shape and postsynaptic firing probabilities...), but rather make a guess at an empirical function.

For now, we just try to emulate the STA dimple by subtracting a gaussian dip from our exponential model of the PSP.

In [26]:
# @time calc_avg_STA(s, p, postsyn_neurons = 1:40, inputs = s.exc_inputs);
@time calc_avg_STA(s, p, 1:40, s.exc_inputs);

  4.951323 seconds (16.24 k allocations: 132.874 MiB, 0.42% gc time)


In [27]:
@time calc_avg_STA_v2(s, p, 1:40, s.exc_inputs);

  4.732793 seconds (53.34 k allocations: 134.654 MiB, 0.35% gc time, 2.02% compilation time)


In [28]:
STAs = (calc_STA(m => n, s, p) for n in 1:40 for m in s.exc_inputs[n]);
@time mean(STAs);

  4.827092 seconds (131.30 k allocations: 148.661 MiB, 0.72% gc time, 1.68% compilation time)


In [23]:
@code_warntype calc_avg_STA(s, p, 1:40, s.exc_inputs);

MethodInstance for VoltoMapSim.calc_avg_STA(::SimData, ::ExperimentParams, ::UnitRange{Int64}, ::ComponentVector{Vector{Int64}})
  from calc_avg_STA(s::SimData, p::ExperimentParams, postsyn_neurons, inputs) in VoltoMapSim at C:\Users\tfiers\phd\pkg\VoltoMapSim\src\sim\post.jl:47
Arguments
  #self#::Core.Const(VoltoMapSim.calc_avg_STA)
  s::SimData
  p::ExperimentParams
  postsyn_neurons::UnitRange{Int64}
  inputs::ComponentVector{Vector{Int64}, Vector{Vector{Int64}}, Tuple{Axis{(exc = 1:800, inh = 801:1000)}}}
Locals
  @_6::Union{Nothing, Tuple{Int64, Int64}}
  avgSTA::Any
  N::Int64
  acc::Union{Nothing, Array}
  @_10::Union{Nothing, Tuple{Int64, Int64}}
  n::Int64
  m::Int64
  STA::Union{Vector, Matrix{Float64}}
Body::Any
1 ──       Core.NewvarNode(:(avgSTA))
│          (acc = VoltoMapSim.nothing)
│          (N = 0)
│    %4  = postsyn_neurons::UnitRange{Int64}
│          (@_6 = Base.iterate(%4))
│    %6  = (@_6 === nothing)::Bool
│    %7  = Base.not_int(%6)::Bool
└───       goto #10 

In [29]:
@code_warntype calc_STA(1 => 1, s, p)

MethodInstance for VoltoMapSim.calc_STA(::Pair{Int64, Int64}, ::SimData, ::ExperimentParams)
  from calc_STA(::Pair{Int64, Int64}, s::SimData, p::ExperimentParams) in VoltoMapSim at C:\Users\tfiers\phd\pkg\VoltoMapSim\src\conntest.jl:19
Arguments
  #self#::Core.Const(VoltoMapSim.calc_STA)
  @_2::Pair{Int64, Int64}
  s::SimData
  p::ExperimentParams
Locals
  @_5::Int64
  to::Int64
  from::Int64
Body::Union{Vector, Matrix{Float64}}
1 ─ %1  = Base.indexed_iterate(@_2, 1)::Core.PartialStruct(Tuple{Int64, Int64}, Any[Int64, Core.Const(2)])
│         (from = Core.getfield(%1, 1))
│         (@_5 = Core.getfield(%1, 2))
│   %4  = Base.indexed_iterate(@_2, 2, @_5::Core.Const(2))::Core.PartialStruct(Tuple{Int64, Int64}, Any[Int64, Core.Const(3)])
│         (to = Core.getfield(%4, 1))
│   %6  = Base.getproperty(s, :signals)::Any
│   %7  = Base.getindex(%6, to)::Any
│   %8  = Base.getproperty(%7, :v)::Any
│   %9  = Base.getproperty(s, :spike_times)::Any
│   %10 = Base.getindex(%9, from)::Any
│   %

In [30]:
@code_warntype calc_STA(s.signals[1].v, s.spike_times[2], p)

MethodInstance for VoltoMapSim.calc_STA(::Vector{Float64}, ::Vector{Float64}, ::ExperimentParams)
  from calc_STA(VI_sig, presynaptic_spikes, p::ExperimentParams) in VoltoMapSim at C:\Users\tfiers\phd\pkg\VoltoMapSim\src\conntest.jl:2
Arguments
  #self#::Core.Const(VoltoMapSim.calc_STA)
  VI_sig::Vector{Float64}
  presynaptic_spikes::Vector{Float64}
  p::ExperimentParams
Locals
  @_5::Union{Nothing, Tuple{Int64, Int64}}
  num_wins::Int64
  win_starts::Vector{Int64}
  STA::Vector{Float64}
  win_size::Int64
  Δt::Float64
  a::Int64
  b::Int64
  @_13::SubArray{Float64, 1, Vector{Float64}, Tuple{UnitRange{Int64}}, true}
Body::Vector{Float64}
1 ─ %1  = Base.getproperty(p, :sim)::VoltoMapSim.SimParams
│   %2  = Base.getproperty(%1, :general)::Any
│   %3  = Base.getproperty(%2, :Δt)::Any
│   %4  = Base.convert(VoltoMapSim.Float64, %3)::Any
│         (Δt = Core.typeassert(%4, VoltoMapSim.Float64))
│   %6  = Base.getproperty(p, :conntest)::VoltoMapSim.ConnTestParams
│   %7  = Base.getproperty(%

In [22]:
# @code_warntype VoltoMapSim.calc_avg_STA_v2(s, p, postsyn_neurons = 1:40, inputs = s.exc_inputs);
@code_warntype VoltoMapSim.calc_avg_STA_v2(s, p, 1:40, s.exc_inputs);

MethodInstance for VoltoMapSim.calc_avg_STA_v2(::SimData, ::ExperimentParams, ::UnitRange{Int64}, ::ComponentVector{Vector{Int64}})
  from calc_avg_STA_v2(s::SimData, p::ExperimentParams, postsyn_neurons, inputs) in VoltoMapSim at C:\Users\tfiers\phd\pkg\VoltoMapSim\src\sim\post.jl:63
Arguments
  #self#::Core.Const(VoltoMapSim.calc_avg_STA_v2)
  s::SimData
  p::ExperimentParams
  postsyn_neurons::UnitRange{Int64}
  inputs::ComponentVector{Vector{Int64}, Vector{Vector{Int64}}, Tuple{Axis{(exc = 1:800, inh = 801:1000)}}}
Locals
  @_6::Union{Nothing, Tuple{Int64, Int64}}
  avgSTA::Vector{Float64}
  N::Int64
  acc::Vector{Float64}
  win_size::Int64
  Δt::Float64
  @_12::Union{Nothing, Tuple{Int64, Int64}}
  n::Int64
  m::Int64
  STA::Union{Vector, Matrix{Float64}}
Body::Vector{Float64}
1 ─       Core.NewvarNode(:(avgSTA))
│   %2  = Base.getproperty(p, :sim)::VoltoMapSim.SimParams
│   %3  = Base.getproperty(%2, :general)::Any
│   %4  = Base.getproperty(%3, :Δt)::Any
│   %5  = Base.convert(V